# Data Aggregation

In [1]:
import numpy as np
import pandas as pd

### Built-in aggregation methods in Pandas  
An aggregation method takes a Series of values and returns a single value.

In [24]:
numbers = pd.Series(np.random.randint(low=1, high=100, size=5))
numbers[2] = np.nan
numbers

0    53.0
1    13.0
2     NaN
3    36.0
4    29.0
dtype: float64

In [26]:
# Some of pandas aggregationmethods are: 

print(f"{numbers.min() = }")
print(f"{numbers.max() = }")
print(f"{numbers.sum() = }")
print(f"{numbers.mean() = }")
print(f"{numbers.sum() / numbers.count() = }")
print(f"{numbers.median() = }")
# print(f"{numbers.mode() = }")
print(f"{numbers.count() = }") # Räknar antalet värden, tar ej med null/none
print(f"{numbers.size = }")  # Antalet rader, (räknar med Null/None)



numbers.min() = 13.0
numbers.max() = 53.0
numbers.sum() = 131.0
numbers.mean() = 32.75
numbers.sum() / numbers.count() = 32.75
numbers.median() = 32.5
numbers.count() = 4
numbers.size = 5


## When run on a dataframe (multiple Series) they return a single value for each Series, forming a new series.

In [37]:
numbers_df = pd.DataFrame(np.random.randint(low=1, high=100, size=[5,5]), columns=["A", "B", "C", "D", "E"])
numbers_df.loc[1:3,"B":"D"] = np.nan
numbers_df

,A,B,C,D,E
0,39,69.0,25.0,46.0,48
1,43,NaN,NaN,NaN,29
2,84,NaN,NaN,NaN,42
3,22,NaN,NaN,NaN,51
4,1,71.0,2.0,72.0,5


In [38]:
numbers_df.min() # Default axis = "index"
numbers_df.min(axis="columns")
numbers_df.min(axis="columns").min()
numbers_df.median(axis="index").median()
numbers_df.median(axis="columns").median()


36.5

In [39]:
numbers_df.isna().sum()

A    0
B    3
C    3
D    3
E    0
dtype: int64

## Working with real data

In [43]:
autos = pd.read_json("../Data/autos.json")
autos.tail()


,aspiration,body-style,bore,city-mpg,compression-ratio,curb-weight,drive-wheels,engine-location,engine-size,engine-type,...,make,normalized-losses,num-of-cylinders,num-of-doors,peak-rpm,price,stroke,symboling,wheel-base,width
200,std,sedan,3.78,23,9.5,2952,rwd,front,141,ohc,...,volvo,95.0,four,four,5400.0,16845.0,3.15,-1,109.1,68.9
201,turbo,sedan,3.78,19,8.7,3049,rwd,front,141,ohc,...,volvo,95.0,four,four,5300.0,19045.0,3.15,-1,109.1,68.8
202,std,sedan,3.58,18,8.8,3012,rwd,front,173,ohcv,...,volvo,95.0,six,four,5500.0,21485.0,2.87,-1,109.1,68.9
203,turbo,sedan,3.01,26,23.0,3217,rwd,front,145,ohc,...,volvo,95.0,six,four,4800.0,22470.0,3.40,-1,109.1,68.9
204,turbo,sedan,3.78,19,9.5,3062,rwd,front,141,ohc,...,volvo,95.0,four,four,5400.0,22625.0,3.15,-1,109.1,68.9


In [42]:
autos["price"].mean()

13207.129353233831

In [45]:
autos[["length", "width", "height"]].head(3)

,length,width,height
0,168.8,64.1,48.8
1,168.8,64.1,48.8
2,171.2,65.5,52.4


In [52]:
# Running mean on multiple columns (DataFrame) returns a Series of means.

autos[["length", "width", "height"]].head(3).mean()
autos[["length", "width", "height"]].mean()

# Same query with different syntax: 
autos[autos["make"]== 'volvo'][["length", "width", "height"]].mean() # For reading and with "loc" also updating/changing values
autos.query("make == 'volvo'")[["length", "width", "height"]].mean() # Only for reading, querying 

length    188.800000
width      67.963636
height     56.236364
dtype: float64

## Multiple aggregation

In [65]:
autos[["length", "width", "height"]].agg(["min", "max", "mean"])

filter_list = [col for col in autos.columns if autos[col].dtype in ["int64", "float64"]]
autos[filter_list].agg(["min", "max", "mean"])

# autos[[col for col in autos.columns if autos[col].dtype in ["int64", "float64"]]].agg(["min", "max", "mean"])
# autos.describe()

,bore,city-mpg,compression-ratio,curb-weight,engine-size,height,highway-mpg,horsepower,length,normalized-losses,peak-rpm,price,stroke,symboling,wheel-base,width
min,2.540000,13.000000,7.000000,1488.000000,61.000000,47.800000,16.00000,48.000000,141.100000,65.0,4150.000000,5118.000000,2.070000,-2.000000,86.600000,60.300000
max,3.940000,49.000000,23.000000,4066.000000,326.000000,59.800000,54.00000,288.000000,208.100000,256.0,6600.000000,45400.000000,4.170000,3.000000,120.900000,72.300000
mean,3.329751,25.219512,10.142537,2555.565854,126.907317,53.724878,30.75122,104.256158,174.049268,122.0,5125.369458,13207.129353,3.255423,0.834146,98.756585,65.907805


### Split-Apply-Combine  
- Select a feature to use as key
- Split the dataset into group for each unique key value
- Apply aggregation to each group
- Combine aggregated data into new dataset  

![image](https://nicholasvadivelu.com/assets/images/posts/groupby/split-apply-combine.svg#center)

### Group by  
Use Pandas .groupby() method to select a key and split into groups.

This creates a new *DataFrameGroupBy* object containing the grouped *DataFrames*

In [66]:
makes = autos.groupby("make")

In [67]:
type(makes)

pandas.core.groupby.generic.DataFrameGroupBy

In [73]:
autos["make"].value_counts()
autos["make"].unique()
autos["make"].nunique()
len(makes)

22

In [76]:
makes.groups
autos.loc[makes.groups["jaguar"]]
makes.get_group("jaguar")

,aspiration,body-style,bore,city-mpg,compression-ratio,curb-weight,drive-wheels,engine-location,engine-size,engine-type,...,make,normalized-losses,num-of-cylinders,num-of-doors,peak-rpm,price,stroke,symboling,wheel-base,width
47,std,sedan,3.63,15,8.1,4066,rwd,front,258,dohc,...,jaguar,145.0,six,four,4750.0,32250.0,4.17,0,113.0,69.6
48,std,sedan,3.63,15,8.1,4066,rwd,front,258,dohc,...,jaguar,NaN,six,four,4750.0,35550.0,4.17,0,113.0,69.6
49,std,sedan,3.54,13,11.5,3950,rwd,front,326,ohcv,...,jaguar,NaN,twelve,two,5000.0,36000.0,2.76,0,102.0,70.6


### Apply & Combine  
It's possible to access a single group as above.   
However, most of the time we rather apply aggregation functions to each group individually and combine the result into a new dataset.


In [84]:
makes.count()
makes["price"].mean()
makes.count().head()
makes[["length","height","length"]].mean().head(3)

,length,height,length
make,,,
alfa-romero,169.600000,50.000000,169.600000
audi,183.828571,54.428571,183.828571
bmw,184.500000,54.825000,184.500000


### SeriesGroupBy


### Multiple aggregation on DataFrameGroupBy  
Using pandas .agg() method on a DataFrameGroupBy to do multiple aggregations on multiple features will return a multi-index column DataFrame.

In [86]:
makes[["length", "width", "height"]].agg(["min", "mean", "max"]).head(3)

length                    width                  height  \
               min        mean    max   min       mean   max    min   
make                                                                  
alfa-romero  168.8  169.600000  171.2  64.1  64.566667  65.5   48.8   
audi         176.6  183.828571  192.7  66.2  68.714286  71.4   52.0   
bmw          176.8  184.500000  197.0  64.8  66.475000  70.9   53.7   

                              
                  mean   max  
make                          
alfa-romero  50.000000  52.4  
audi         54.428571  55.9  
bmw          54.825000  56.3

### Custom columns aggregation 

In [88]:
autos.groupby("make").agg({"price" : "mean", "horsepower" : "max"}).head(3)

,price,horsepower
make,,
alfa-romero,15498.333333,154.0
audi,17859.166667,160.0
bmw,26118.750000,182.0


In [94]:
autos.groupby("make").agg(
    average_price = pd.NamedAgg(column="price", aggfunc="mean"),
    max_horsepower = pd.NamedAgg(column="horsepower", aggfunc="max"),
    min_horsepower = pd.NamedAgg(column="horsepower", aggfunc="min"),
    ).head(3)

,average_price,max_horsepower,min_horsepower
make,,,
alfa-romero,15498.333333,154.0,111.0
audi,17859.166667,160.0,102.0
bmw,26118.750000,182.0,101.0
